#Download dataset

##linux

In [ ]:
!rm -rf *
!git clone "https://github.com/hmda77/Ensemble-Indoor-Loc"
!cp /content/Ensemble-Indoor-Loc/JUIndoorLoc/JUIndoorLoc-Test-data.csv /content/
!cp /content/Ensemble-Indoor-Loc/JUIndoorLoc/JUIndoorLoc-Training-data.csv /content/
!rm -rf /content/Ensemble-Indoor-Loc/

rm: cannot remove 'drive/MyDrive': Operation canceled
rm: cannot remove 'drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove 'drive/.file-revisions-by-id': Operation canceled
rm: cannot remove 'drive/.Trash-0': Directory not empty
Cloning into 'Ensemble-Indoor-Loc'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 56 (delta 17), reused 5 (delta 1), pack-reused 0
Receiving objects: 100% (56/56), 695.75 KiB | 893.00 KiB/s, done.
Resolving deltas: 100% (17/17), done.


# windows

In [ ]:
!cd ./Desktop/Projects/ && git clone "https://github.com/hmda77/Ensemble-Indoor-Loc"
!cd ./Desktop/Projects/Ensemble-Indoor-Loc/


/bin/bash: line 1: cd: ./Desktop/Projects/: No such file or directory
/bin/bash: line 1: cd: ./Desktop/Projects/Ensemble-Indoor-Loc/: No such file or directory


# Import Packages

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder , normalize
from sklearn.model_selection import ShuffleSplit
from itertools import cycle

import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

from matplotlib import pyplot

# Metric Functions


In [ ]:

# Make the confusion matrix
def confusion_plot(y_test,y_pred,title):
    cmt = confusion_matrix(y_test,y_pred)
    plt.rcParams['figure.figsize'] = (10,8)
    sns.heatmap(cmt,fmt='',annot=True,linewidth=0.01,cmap=sns.cubehelix_palette(as_cmap=True))
    plt.title("confusion matrix {}".format(title))
    plt.xlabel("predicted")
    plt.ylabel("true")
    plt.show()


# generate classification report
def generate_classification_report(y_test,y_pred):
    report=pd.DataFrame.from_dict(classification_report(y_pred,y_test,output_dict=True)).T
    # report['Label']=[data_classes[i]  if i<5 else " " for i,x in enumerate(report.index)]
    report=report[['f1-score','precision','recall','support']]
    pd.set_option('display.max_rows', report.shape[0]+1)
    return report

# ROC Curve
def plot_roc_curve(y_test,proba,pos_label):
    fpr, tpr, thresholds = roc_curve(y_test, proba[:,pos_label], pos_label= pos_label)
    roc_auc = roc_auc_score(y_test, proba, multi_class='ovr')

    plt.rcParams['figure.figsize'] = (5,4)
    plt.plot(fpr, tpr,'k:',lw=5, label='average ROC curve (area = {0:0.2f})'.format(roc_auc))
    plt.plot([0 ,1],[0 ,1],'r')
    plt.legend(loc="lower right")
    # plt.title('ROC Curve for calss {0}'.format(data_classes[pos_label]))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()


# ROC Curve for all classes
def plot_roc_curve_all(y_test,proba,n_class,title):
    for i in range(n_class):
      fpr, tpr, thresholds = roc_curve(y_test, proba[:,i],pos_label= i)
      plt.plot(fpr, tpr,lw=1)
    plt.plot([0 ,1],[0 ,1],'r')
    plt.title('ROC Curve for all classes in {}'.format(title))
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    # plt.legend(data_classes)
    plt.show()

#Dateset Proccesses

In [ ]:
data_train = pd.read_csv("C:\\Users\Hamid\content\JUIndoorLoc-Training-data.csv")
data_train.head(5)
data_test = pd.read_csv("C:\\Users\Hamid\content\JUIndoorLoc-Test-data.csv")
data_test.head(5)

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:3: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:3: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Hamid\AppData\Local\Temp\ipykernel_2816\2129459651.py:1: SyntaxWarning: invalid escape sequence '\H'
  data_train = pd.read_csv("C:\\Users\Hamid\content\JUIndoorLoc-Training-data.csv")
C:\Users\Hamid\AppData\Local\Temp\ipykernel_2816\2129459651.py:3: SyntaxWarning: invalid escape sequence '\H'
  data_test = pd.read_csv("C:\\Users\Hamid\content\JUIndoorLoc-Test-data.csv")


,Cid,AP001,AP002,AP003,AP004,AP005,AP006,AP007,AP008,AP009,...,AP167,AP168,AP169,AP170,AP171,AP172,Rs,Hpr,Did,Ts
0,L4-33-13,-77,-58,-66,-64,-92,-66,-66,-93,-93,...,-110,-110,-110,-110,-110,-110,0,0,D2,1489813137748
1,L4-33-13,-90,-58,-78,-56,-92,-74,-74,-87,-93,...,-110,-110,-110,-110,-110,-110,0,0,D2,1489813179138
2,L4-33-13,-80,-64,-78,-56,-92,-74,-74,-87,-93,...,-110,-110,-110,-110,-110,-110,0,0,D2,1489812948443
3,L4-33-13,-72,-60,-74,-58,-93,-75,-76,-95,-93,...,-110,-110,-110,-110,-110,-110,0,0,D2,1489812959103
4,L4-33-13,-82,-56,-74,-56,-93,-71,-76,-89,-110,...,-110,-110,-110,-110,-110,-110,0,0,D2,1489813079167


In [ ]:
frames = [data_train, data_test]
mergedDataset = pd.concat(frames)
mergedDataset.head(5)

,Cid,AP001,AP002,AP003,AP004,AP005,AP006,AP007,AP008,AP009,...,AP167,AP168,AP169,AP170,AP171,AP172,Rs,Hpr,Did,Ts
0,L4-40-1,-84,-80,-71,-58,-110,-72,-71,-110,-110,...,-110,-110,-110,-110,-110,-110,0,1,D4,1469870570949
1,L4-40-1,-84,-79,-71,-58,-110,-72,-71,-110,-110,...,-110,-110,-110,-110,-110,-110,0,1,D4,1470047205646
2,L4-40-1,-110,-110,-70,-56,-110,-69,-68,-110,-110,...,-110,-110,-110,-110,-110,-110,0,1,D4,1469870932338
3,L4-40-1,-110,-110,-70,-53,-110,-69,-68,-110,-110,...,-110,-110,-110,-110,-110,-110,0,1,D4,1470047629440
4,L4-37-2,-84,-82,-75,-65,-110,-73,-75,-110,-110,...,-110,-110,-110,-110,-110,-110,0,1,D4,1469876622694


In [ ]:
mergedDataset['Did'] = mergedDataset['Did'].astype(str).str[1]
mergedDataset['Did'] = pd.to_numeric(mergedDataset['Did'])

In [ ]:
#make X_train from Tr
X = mergedDataset.loc[:, (mergedDataset.columns != 'Ts') & (mergedDataset.columns != 'Cid')]
y = mergedDataset.Cid



le = LabelEncoder()
le.fit(y)
le.transform(y)


X_train = X
y_train_encode =  le.transform(y)


In [ ]:
print(len(np.unique(y_train_encode)))
print(len(y.unique()))

991
991


# 5 Fold CrossValidation

In [ ]:

base_estimator = DecisionTreeClassifier(max_depth=50)  # Shallow decision tree as a weak learner
n_estimators = 100  # Number of weak learners in the ensemble
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=n_estimators)

scores = cross_val_score(adaboost, X_train, y_train_encode, cv=5)

C:\Users\Hamid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
scores

array([0.95584467, 0.64419476, 0.86595703, 0.94914252, 0.62756309])

# 5 Fold CrossValidation with shuffle

In [ ]:

base_estimator = DecisionTreeClassifier(max_depth=50)  # Shallow decision tree as a weak learner
n_estimators = 100  # Number of weak learners in the ensemble
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=n_estimators)
cv = ShuffleSplit(n_splits=5, test_size=0.15, random_state=42)
scores = cross_val_score(adaboost, X_train, y_train_encode, cv=cv)

In [ ]:
scores

array([0.96425756, 0.9695138 , 0.97056505, 0.97345598, 0.96636005])